In [44]:
import numpy as np
import pandas as pd
import pickle 
from sklearn.feature_extraction.text import CountVectorizer as cv
from sklearn.metrics.pairwise import cosine_similarity as cs

In [45]:
data = pd.read_csv("/kaggle/input/acm-dattaset/tmdb-movies (1).csv")
data.shape
data.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/15,5562,6.5,2015,1.379999e+08,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/15,6185,7.1,2015,1.379999e+08,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/15,2480,6.3,2015,1.012000e+08,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/15,5292,7.5,2015,1.839999e+08,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/15,2947,7.3,2015,1.747999e+08,1.385749e+09


In [46]:
data.sort_values(by=['id'])
data['keywords']

0        monster|dna|tyrannosaurus rex|velociraptor|island
1         future|chase|post-apocalyptic|dystopia|australia
2        based on novel|revolution|dystopia|sequel|dyst...
3                    android|spaceship|jedi|space opera|3d
4                      car race|speed|revenge|suspense|car
                               ...                        
10861                             surfer|surfboard|surfing
10862                            car race|racing|formula 1
10863                             car|trolley|stealing car
10864                                                spoof
10865                  fire|gun|drive|sacrifice|flashlight
Name: keywords, Length: 10866, dtype: object

In [47]:
data.describe()

,id,popularity,budget,revenue,runtime,vote_count,vote_average,release_year,budget_adj,revenue_adj
count,10866.000000,10866.000000,1.086600e+04,1.086600e+04,10866.000000,10866.000000,10866.000000,10866.000000,1.086600e+04,1.086600e+04
mean,66064.177434,0.646441,1.462570e+07,3.982332e+07,102.070863,217.389748,5.974922,2001.322658,1.755104e+07,5.136436e+07
std,92130.136561,1.000185,3.091321e+07,1.170035e+08,31.381405,575.619058,0.935142,12.812941,3.430616e+07,1.446325e+08
min,5.000000,0.000065,0.000000e+00,0.000000e+00,0.000000,10.000000,1.500000,1960.000000,0.000000e+00,0.000000e+00
25%,10596.250000,0.207583,0.000000e+00,0.000000e+00,90.000000,17.000000,5.400000,1995.000000,0.000000e+00,0.000000e+00
50%,20669.000000,0.383856,0.000000e+00,0.000000e+00,99.000000,38.000000,6.000000,2006.000000,0.000000e+00,0.000000e+00
75%,75610.000000,0.713817,1.500000e+07,2.400000e+07,111.000000,145.750000,6.600000,2011.000000,2.085325e+07,3.369710e+07
max,417859.000000,32.985763,4.250000e+08,2.781506e+09,900.000000,9767.000000,9.200000,2015.000000,4.250000e+08,2.827124e+09


In [48]:
data.columns

Index(['id', 'imdb_id', 'popularity', 'budget', 'revenue', 'original_title',
       'cast', 'homepage', 'director', 'tagline', 'keywords', 'overview',
       'runtime', 'genres', 'production_companies', 'release_date',
       'vote_count', 'vote_average', 'release_year', 'budget_adj',
       'revenue_adj'],
      dtype='object')

In [49]:
data['delim'] = data['genres']+data['tagline']+data['original_title']+data['keywords']+data['overview']
data = data[['id', 'original_title', 'delim', 'homepage']]
data.head()

,id,original_title,delim,homepage
0,135397,Jurassic World,Action|Adventure|Science Fiction|ThrillerThe p...,http://www.jurassicworld.com/
1,76341,Mad Max: Fury Road,Action|Adventure|Science Fiction|ThrillerWhat ...,http://www.madmaxmovie.com/
2,262500,Insurgent,Adventure|Science Fiction|ThrillerOne Choice C...,http://www.thedivergentseries.movie/#insurgent
3,140607,Star Wars: The Force Awakens,Action|Adventure|Science Fiction|FantasyEvery ...,http://www.starwars.com/films/star-wars-episod...
4,168259,Furious 7,Action|Crime|ThrillerVengeance Hits HomeFuriou...,http://www.furious7.com/


In [50]:
count_delims = cv(max_features=10866, stop_words='english')

In [51]:
vector = count_delims.fit_transform(data['delim'].values.astype('U')).toarray()
vector.shape

(10866, 10866)

In [52]:
match_values = cs(vector)
match_values

array([[1.        , 0.13025651, 0.10147651, ..., 0.        , 0.05564149,
        0.        ],
       [0.13025651, 1.        , 0.09567297, ..., 0.        , 0.13770566,
        0.        ],
       [0.10147651, 0.09567297, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.05564149, 0.13770566, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [53]:
def top_matching(movie_name, num_of_recommendations): 
    list_res = []
    idx = data[data['original_title'] == movie_name].index[0]
    distance = sorted(list(enumerate(match_values[idx])), reverse = True, key = lambda vector:vector[1])
    for i in distance[1:num_of_recommendations+1]:
        list_res.append(data.iloc[i[0]]['original_title'])
    return list_res 

In [54]:
print(top_matching("The Godfather", 10))

['The Godfather: Part II', 'Mambo Italiano', 'City Island', 'C.R.A.Z.Y.', 'Lullaby', 'All Good Things', 'The Family', 'This Christmas', 'Once Were Warriors', 'The Last Song']


In [55]:
pickle.dump(data, open('movies_list.mkl', 'wb'))
pickle.dump(match_values, open('cosine_similarities.mkl', 'wb'))


In [56]:
pickle.load(open('movies_list.mkl', 'rb'))

,id,original_title,delim,homepage
0,135397,Jurassic World,Action|Adventure|Science Fiction|ThrillerThe p...,http://www.jurassicworld.com/
1,76341,Mad Max: Fury Road,Action|Adventure|Science Fiction|ThrillerWhat ...,http://www.madmaxmovie.com/
2,262500,Insurgent,Adventure|Science Fiction|ThrillerOne Choice C...,http://www.thedivergentseries.movie/#insurgent
3,140607,Star Wars: The Force Awakens,Action|Adventure|Science Fiction|FantasyEvery ...,http://www.starwars.com/films/star-wars-episod...
4,168259,Furious 7,Action|Crime|ThrillerVengeance Hits HomeFuriou...,http://www.furious7.com/
...,...,...,...,...
10861,21,The Endless Summer,NaN,NaN
10862,20379,Grand Prix,Action|Adventure|DramaCinerama sweeps YOU into...,NaN
10863,39768,Beregis Avtomobilya,NaN,NaN
10864,21449,"What's Up, Tiger Lily?",Action|ComedyWOODY ALLEN STRIKES BACK!What's U...,NaN


In [57]:
pickle.load(open('cosine_similarities.mkl', 'rb'))

array([[1.        , 0.13025651, 0.10147651, ..., 0.        , 0.05564149,
        0.        ],
       [0.13025651, 1.        , 0.09567297, ..., 0.        , 0.13770566,
        0.        ],
       [0.10147651, 0.09567297, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.05564149, 0.13770566, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])